In [38]:
import pickle
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split, Subset
# import torchvision
import matplotlib.pyplot as plt
import sinabs
import sinabs.activation
import sinabs.layers as sl
from sinabs.from_torch import from_model
import os
import shutil
import samna
from samna.unifirm.timestamp import StopWatch
from sklearn.model_selection import KFold
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)
torch.cuda.empty_cache()

True


In [39]:
class MyData(Dataset):
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.data_path = os.listdir(self.root_dir)

    # calculate the new matrix by suming every n columns
    def sum_adjacent_cols(self, matrix, n):
        num_cols = matrix.shape[1]
        result = torch.zeros((matrix.shape[0],int(np.ceil(num_cols/n))))
        for i in range(n):
            result += matrix[:, i:num_cols:n]
        return result
    
    def __getitem__(self, idx):
        data_name = self.data_path[idx]
        data_item_path = os.path.join(self.root_dir, data_name)
        with open(data_item_path, 'rb') as f:
            data = np.load(f)

        data = torch.from_numpy(data).float()   # [256, 2000]
        # data = torch.transpose(data, 0, 1)
        data = self.sum_adjacent_cols(data, 10) # [256, 200]
        data = torch.transpose(data, 0, 1)      # [200, 256]
        data = data.view(200, 16, 16)           # [200, 16, 16]
        data = data.unsqueeze(1)                # [200, 1, 16, 16]

        if data_name[-6] == '_':
            label = torch.tensor(eval(data_name[-5]), dtype=torch.long)
        else:
            label = torch.tensor(eval(data_name[-6:-4]), dtype=torch.long)
        
        return data, label

    def __len__(self):
        return len(self.data_path)

root_dir_1 = '/home/xxc/PhD/synsense/Data/mg_tap_9positions/deep_learning/train'
root_dir_2 = '/home/xxc/PhD/synsense/Data/mg_tap_9positions/deep_learning/test'

train_data = MyData(root_dir_1)
test_data  = MyData(root_dir_2)

print(train_data[0][0].shape)
print(len(train_data), len(test_data))

torch.Size([200, 1, 16, 16])
180 90


Training

In [40]:
lr = 1e-2

device = 'cuda:0'
k_folds = 3
kfold = KFold(n_splits=k_folds, shuffle=False)
batch_size = 30
time_steps = 200
loss_fn = nn.CrossEntropyLoss()

fold_train_acc = []
fold_val_acc = []

for fold, (train_ids, val_ids) in enumerate(kfold.split(train_data)):
    # 模型初始化
    linear_model = nn.Sequential(
        nn.Conv2d(1, 8, 3, 1, bias=False),
        nn.ReLU(),
        nn.AvgPool2d(2, 2),
        nn.Conv2d(8, 16, 3, 1, bias=False),
        nn.ReLU(),
        # nn.AvgPool2d(2, 2),
        nn.Conv2d(16, 32, 3, 1, bias=False),
        nn.ReLU(),
        nn.Flatten(),
        nn.Linear(288, 500, bias=False),
        nn.ReLU(),
        nn.Linear(500, 9, bias=False)
    )
    linear_model = from_model(linear_model, batch_size=batch_size, input_shape=(1, 16, 16), 
                        add_spiking_output=True, synops=False, num_timesteps=time_steps).to(device)
    optimizer = torch.optim.Adam(linear_model.parameters(), lr)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)
    # 分割数据集
    train_sub = Subset(train_data, train_ids)
    val_sub = Subset(train_data, val_ids)
    # 创建数据加载器
    train_loader = DataLoader(train_sub, batch_size, shuffle=True, drop_last=True)
    val_loader  = DataLoader(val_sub, batch_size, drop_last=True)

    # 训练模型
    linear_model.train()
    epochs = 8
    for e in range(epochs):
        running_loss = 0.
        acc = 0
        scheduler.step()
        for i, (input, target) in enumerate(train_loader):
            optimizer.zero_grad()
            linear_model.reset_states()

            input = input.to(device)
            input = input.view(batch_size*time_steps, 1, 16, 16)
            target = target.to(device)
            output = linear_model(input)
            output = output.view(batch_size, time_steps, 9)
            sum_output = output.sum(1)
            loss = loss_fn(sum_output, target)
            loss.backward()
            optimizer.step()

            running_loss += loss

            for j in range(batch_size):
                if sum_output[j].argmax() == target[j]:
                    acc += 1

        print("epoch: %d, accuracy: %.2f%%, running_loss: %.2f, current_lr: %.6f" 
              % (e, acc/len(train_sub)*100, running_loss, scheduler.get_last_lr()[0]) )
    fold_train_acc.append(np.around(acc/len(train_sub)*100, 2))

    # 验证模型
    acc_num = 0
    for i, (data, target) in enumerate(val_loader):
        with torch.no_grad():
            linear_model.reset_states()
            data = data.to(device)
            data = data.view(batch_size*time_steps, 1, 16, 16)
            target = target.to(device)
            output = linear_model(data)
            output = output.view(batch_size, time_steps, 9)
            sum_output = output.sum(1)

        for j in range(batch_size):
            if sum_output[j].argmax() == target[j]:
                acc_num += 1
    print("accuracy on validation set: %.2f%%" % (acc_num/len(val_sub)*100))
    fold_val_acc.append(np.around(acc_num/len(val_sub)*100, 2))

epoch: 0, accuracy: 20.83%, running_loss: 26.53, current_lr: 0.010000
epoch: 1, accuracy: 47.50%, running_loss: 7.73, current_lr: 0.010000
epoch: 2, accuracy: 60.00%, running_loss: 4.72, current_lr: 0.010000
epoch: 3, accuracy: 66.67%, running_loss: 3.52, current_lr: 0.010000
epoch: 4, accuracy: 90.83%, running_loss: 1.26, current_lr: 0.001000
epoch: 5, accuracy: 92.50%, running_loss: 0.72, current_lr: 0.001000
epoch: 6, accuracy: 92.50%, running_loss: 0.69, current_lr: 0.001000
epoch: 7, accuracy: 92.50%, running_loss: 0.69, current_lr: 0.001000
accuracy on validation set: 81.67%
epoch: 0, accuracy: 20.83%, running_loss: 35.31, current_lr: 0.010000
epoch: 1, accuracy: 24.17%, running_loss: 11.93, current_lr: 0.010000
epoch: 2, accuracy: 34.17%, running_loss: 6.92, current_lr: 0.010000
epoch: 3, accuracy: 63.33%, running_loss: 5.01, current_lr: 0.010000
epoch: 4, accuracy: 67.50%, running_loss: 4.83, current_lr: 0.001000
epoch: 5, accuracy: 65.00%, running_loss: 3.65, current_lr: 0.001

In [41]:
print(fold_train_acc)
print(fold_val_acc)

sum = 0
cnt = 0
for i in fold_train_acc:
    sum += i
    cnt += 1
print(sum/cnt)

sum = 0
cnt = 0
for i in fold_val_acc:
    sum += i
    cnt += 1
print(sum/cnt)

[92.5, 93.33, 95.83]
[81.67, 100.0, 100.0]
93.88666666666666
93.89


In [42]:
model_path = './models/9pos-scnn.pth'
torch.save(linear_model, model_path)

Testing

In [43]:
model = torch.load(model_path)
test_loader  = DataLoader(test_data, batch_size, drop_last=True)

acc_num = 0
model.eval()
for i, (data, target) in enumerate(test_loader):
    with torch.no_grad():
        model.reset_states()
        data = data.to(device)
        data = data.view(batch_size*time_steps, 1, 16, 16)
        target = target.to(device)
        output = model(data)
        output = output.view(batch_size, time_steps, 9)
        sum_output = output.sum(1)

    for j in range(batch_size):
        if sum_output[j].argmax() == target[j]:
            acc_num += 1
print("accuracy on testing set: %.2f%%" % (acc_num/len(test_data)*100))

accuracy on testing set: 100.00%
